In [1]:
import pymongo
import time
import pandas as pd
import json
import nltk

# Conexão com o MongoDB

In [2]:
client = pymongo.MongoClient()

# database
db = client['8M2020_samples']  

# collection
collection = db.tweets_es

In [3]:
pipeline = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [4]:
time_start = time.time()
document = list(collection.aggregate(pipeline=pipeline))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
print(f'{len(document)} documentos selecionados')

Tempo de execução: 00:00:44
1898572 documentos selecionados


In [5]:
print(f'Amostra de um documento:\n')
print(json.dumps(document[0], sort_keys=False, indent=3, ensure_ascii=False))

Amostra de um documento:

{
   "tweet_id": "1236411302286888963",
   "tweet_text": "Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México.\n\n#UnDíaSinMujeres será el 9 de marzo (#9M)\n\n➡️ https://t.co/gD9aWLYTAf https://t.co/vPzEwHoN7h",
   "num_likes": 0,
   "num_retweets": 0,
   "quoted_status_id": null,
   "retweeted_status_id": null,
   "reply_to_user": null,
   "reply_to_status": null,
   "user_id": "1133175210696429568",
   "screen_name": "DatamosMx",
   "followers": 1331,
   "following": 41,
   "date": "07/03/2020 22:00:00"
}


In [6]:
df = pd.DataFrame(document)
df = df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]
df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1236411302286888963,07/03/2020 22:00:00,Rumbo al #8M | Usuarios de redes sociales conv...,0,0,None,None,None,None,1133175210696429568,DatamosMx,1331,41
1,1236411306334273536,07/03/2020 22:00:01,La USAC llegando al espacio una cátedra de gén...,0,0,None,1236140760761802752,None,None,290815733,ravarick,1129,1773
2,1236411302504992771,07/03/2020 22:00:00,Cuando Norma tenía 19 años fue vendida a un ba...,0,64,1236403803521527809,None,None,None,1921923744,ZBaqueiro,77,119
3,1236411301154500610,07/03/2020 22:00:00,"""El Estado mexicano ha ignorado demasiados año...",2,0,None,None,None,None,918902439801118720,muraldegenero,551,562
4,1236411305638219776,07/03/2020 22:00:01,❌NO PUEDEN\n❌NO PUEDEN\n❌NO PUEDEN\n\n❌ CLARO ...,0,10,1236204966236258304,None,None,None,939976606180892673,Guesaso1,560,655


In [7]:
print(f'Amostra de um tweet antes do pré-processamento:\n\n{df.tweet_text[0]}')

Amostra de um tweet antes do pré-processamento:

Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México.

#UnDíaSinMujeres será el 9 de marzo (#9M)

➡️ https://t.co/gD9aWLYTAf https://t.co/vPzEwHoN7h


In [8]:
df.to_csv('tweets_es.csv')

### Pré-processamento

In [9]:
# remover links
df['clean_text'] = df['tweet_text'].str.replace(r"http\S+","") 

df.clean_text[0]

'Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México.\n\n#UnDíaSinMujeres será el 9 de marzo (#9M)\n\n➡️  '

In [10]:
# remover mentions
df['clean_text'] = df['clean_text'].str.replace(r"@\S+","")

df.clean_text[0]

'Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México.\n\n#UnDíaSinMujeres será el 9 de marzo (#9M)\n\n➡️  '

In [11]:
# remover hashtags
# df['clean_text'] = df['clean_text'].str.replace(r"#\S+"," ")

# df.clean_text[0]

In [12]:
# remover quebra de linhas
df['clean_text'] = df['clean_text'].str.replace(r"\n"," ")

df.clean_text[0]

'Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México.  #UnDíaSinMujeres será el 9 de marzo (#9M)  ➡️  '

In [13]:
# remover pontuações
df['clean_text'] = df['clean_text'].str.replace(r"[^\w\s#]|_"," ")

df.clean_text[0]

'Rumbo al #8M   Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México   #UnDíaSinMujeres será el 9 de marzo  #9M       '

In [14]:
# remover números
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\b(?:[0-9]*)\b","")

df.clean_text[0]

'Rumbo al #8M   Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México   #UnDíaSinMujeres será el  de marzo  #9M'

In [15]:
# remover espaços duplos
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\s{2,}"," ")

df.clean_text[0]

'Rumbo al #8M Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en México #UnDíaSinMujeres será el de marzo #9M'

### converter todas as letras para minúsculas

In [16]:
df['clean_text'] = df['clean_text'].str.lower()

df.clean_text[0]

'rumbo al #8m usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en méxico #undíasinmujeres será el de marzo #9m'

### remover letras que se repetem em sequência mais de 3x

In [17]:
import string
alphabet = list(string.ascii_lowercase)

for letter in alphabet:
    pattern = '{}{}'.format(letter, '{3,}')
    df['clean_text'] = df['clean_text'].str.replace(r'{}'.format(pattern), letter)
    
df.clean_text[0]

'rumbo al #8m usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en méxico #undíasinmujeres será el de marzo #9m'

### remover stpwords

In [19]:
with open('../stopwords/spanish.txt', 'r') as file:
    stopwords = file.readlines()
    stopwords = [sw.replace('\n','') for sw in stopwords]

def remover_stopwords(texto):
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_stopwords)

df.clean_text[0]

'rumbo #8m usuarios redes sociales convocaron paro nacional protesta aumento violencia mujeres méxico #undíasinmujeres marzo #9m'

### remover palavras com 2 caracteres ou menos

In [20]:
def remover_len_2(texto):
    palavras = [i for i in texto.split() if len(i) > 2]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_len_2)

df.clean_text[0]

'rumbo #8m usuarios redes sociales convocaron paro nacional protesta aumento violencia mujeres méxico #undíasinmujeres marzo #9m'

In [21]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

rumbo #8m usuarios redes sociales convocaron paro nacional protesta aumento violencia mujeres méxico #undíasinmujeres marzo #9m


In [22]:
def reduzir_radical(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = [stemmer.stem(i) for i in texto.split()]
    return (" ".join(palavras))

In [23]:
reduzir_radical(df.tweet_text[0])

'rumb al #8m | usuari de red soc convocaron a un par nacion com protest por el aument de la violenc en contr de muj en méxico. #undíasinmuj ser el 9 de marz (#9m) ➡️ https://t.co/gd9awlytaf https://t.co/vpzewhon7h'

In [24]:
# df['tweet_text'] = df['tweet_text'].apply(reduzir_radical)

In [25]:
import unicodedata

def remover_acentuacao(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return (palavra_sem_acento)

In [26]:
remover_acentuacao(df.tweet_text[0])

'Rumbo al #8M | Usuarios de redes sociales convocaron a un paro nacional como protesta por el aumento de la violencia en contra de mujeres en Mexico.\n\n#UnDiaSinMujeres sera el 9 de marzo (#9M)\n\n➡️ https://t.co/gD9aWLYTAf https://t.co/vPzEwHoN7h'

In [27]:
df['clean_text'] = df['clean_text'].apply(remover_acentuacao)

In [28]:
# remover non ascii caracteres
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"[^a-zA-z0-9#|\s]","")

df.clean_text[0]

'rumbo #8m usuarios redes sociales convocaron paro nacional protesta aumento violencia mujeres mexico #undiasinmujeres marzo #9m'

In [29]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

rumbo #8m usuarios redes sociales convocaron paro nacional protesta aumento violencia mujeres mexico #undiasinmujeres marzo #9m


### Etapa opcional

O atributo 'lang' presentes nos documentos podem ter sido atribuidos erroneamente. Objetivando eliminar tweets que possam ser de um idioma não requisitado, desenvolveram-se 2 funções para ajudar a confirmar o idioma destes.

Para desabilitar warnings:

In [30]:
# import logging
# logging.disable(logging.WARNING)

#### 1. Usando polyglot

In [31]:
# import polyglot
# from polyglot.detect import Detector

In [32]:
# def get_polyglot_language(text):
#     try:
#         detector = Detector(text)
#     except polyglot.detect.base.UnknownLanguage:
#         lang = 'Undefined'
#     else:
#         lang = detector.language.name
    
#     return (lang)

In [33]:
# get_polyglot_language(df['tweet_text'][0])

In [34]:
# df['polyglot_lang'] = df['tweet_text'].apply(get_polyglot_language)

In [35]:
# df['polyglot_lang'].value_counts()

#### 2. Usando textblob

In [36]:
# import textblob
# from textblob import TextBlob
# import pycountry

In [37]:
# def get_textblob_language(text):
#     try:
#         time.sleep(0.2)
#         b = TextBlob(text)
#         language = pycountry.languages.get(alpha_2=b.detect_language())
#         language_name = language.name
#     except textblob.exceptions.TranslatorError:
#         # language_name = "Deu ruim"
#         language_name = 'Undefined'
    
#     return(language_name) 

In [38]:
# get_textblob_language(df['tweet_text'][0])

In [39]:
# df['textblob_lang'] = df['tweet_text'].apply(get_textblob_language)

Para reabilitar warnings:

In [40]:
# logging.disable(logging.NOTSET)

In [41]:
# df.shape

# Criação do arquivo .csv

### Cenário 1: base completa

Caso tenha se utilizado umas das duas funções - **get_polyglot_language**, por exemplo - use o seguinte trecho para gerar o arquivo .csv:

In [42]:
# index = df['polyglot_lang'] == 'Portuguese'
# new_df = df[index]
# new_df = new_df.drop('polyglot_lang', axis=1)

In [43]:
# new_df.to_csv('cenario1_8M2020_polyglot_tweets_pt.csv', index=False)

Caso contrário:

In [44]:
# df = df.drop('polyglot_lang', axis=1)

In [45]:
df = df[df['clean_text'] != '']

In [46]:
df.shape

(1897848, 14)

In [47]:
df.to_csv('cenario1_8M2020_tweets_es.csv', index=False)

criação do arquivo .txt (somente com os textos dos tweets):

In [48]:
import numpy as np

In [49]:
tweets_text = np.array(df['clean_text'])

In [50]:
np.savetxt(fname='cenario1_8M2020_tweets_es.txt', X=tweets_text, fmt='%s')

## Cenário 2: tweets sem duplicatas

In [51]:
df.drop_duplicates(['clean_text'], inplace = True)

In [52]:
df.shape

(309082, 14)

In [53]:
df.to_excel('exemplo.xlsx')

In [54]:
df.to_csv('cenario2_8M2020_tweets_es.csv', index=False) 

In [55]:
tweets_text = np.array(df['clean_text'])

In [56]:
np.savetxt(fname='cenario2_8M2020_tweets_es.txt', X=tweets_text, fmt='%s')